In [ ]:
import GEOparse
gse = GEOparse.get_GEO("GSE197406", destdir= "../GEO")
gpl = GEOparse.get_GEO("GSE197406", destdir= "../GEO")

In [ ]:
len(gse.gsms)
list(gse.gsms.keys())[ :3]

In [ ]:
gsm_keys = list(gse.gsms.keys())
example = gse.gsms[gsm_keys[0]]
print(example.table.head())

In [ ]:
expr_matrix = gse.pivot_samples("VALUE")
expr_matrix.head(3)

In [ ]:
pheno = gse.phenotype_data
pheno.columns

In [102]:
# Create folder path if not already existing
import os
os.makedirs("../data/GSE", exist_ok=True)
expr_matrix.to_csv("../data/GSE/GSE197406_expression_matrix.csv")
pheno.to_csv("../data/GSE/GSE197406_pheno_data.csv")

In [ ]:
import GEOparse

# Load the platform used by GSE197406
gpl = GEOparse.get_GEO("GPL570", destdir="../GEO")

print("✅ Platform loaded:", gpl.metadata["title"][0])
print("Number of probe annotations:", len(gpl.table))


In [ ]:
# Extract relevant columns
gpl_annot = gpl.table[["ID", "Gene Symbol", "Gene Title"]]
gpl_annot.head()


In [ ]:
# Make sure expr index matches probe IDs
expr_matrix.index.name = "ID"

# Merge probe annotation with expression data
expr_annot = gpl_annot.merge(expr_matrix, on="ID", how="inner")

print("✅ Annotated expression matrix shape:", expr_annot.shape)
expr_annot.head()


In [ ]:
# Extract table with probe ID → gene symbol
gpl_table = gpl.table[["ID", "Gene Symbol"]].copy()
gpl_table.columns = ["probe_id", "gene_symbol"]
gpl_table.head()


In [107]:
# Add gene symbols to expression matrix
expr_annot = expr_matrix.merge(gpl_table, left_index=True, right_on="probe_id")

# Drop duplicates (some probes map to same gene)
expr_annot = expr_annot.drop_duplicates(subset="gene_symbol").set_index("gene_symbol")

expr_annot.head()


,GSM5916425,GSM5916426,GSM5916427,GSM5916428,GSM5916429,GSM5916430,GSM5916431,GSM5916432,GSM5916433,GSM5916434,GSM5916435,GSM5916436,GSM5916437,GSM5916438,GSM5916439,probe_id
gene_symbol,,,,,,,,,,,,,,,,
DDR1 /// MIR4640,8.25153,7.76315,8.40313,7.98926,8.05217,8.44563,7.58667,8.02614,9.79436,9.66555,9.49759,9.89471,8.45370,8.85845,8.84316,1007_s_at
RFC2,6.74268,6.75010,6.50660,6.88312,6.97066,6.76910,6.89180,6.67075,6.93478,6.90036,6.62057,6.62056,6.52978,6.72519,6.55780,1053_at
HSPA6,6.74961,6.53426,6.72746,6.90891,6.80198,6.80546,6.76402,6.76341,6.74732,6.86814,6.90635,6.84800,6.63013,6.73454,6.55409,117_at
PAX8,8.79223,8.73107,9.32590,9.22579,9.28703,9.42747,9.04310,8.86239,9.51909,9.22565,9.39310,9.32185,9.42897,8.99391,8.99834,121_at
GUCA1A,5.60172,5.69342,5.70023,5.77163,5.73856,5.75393,5.76626,5.64828,5.76027,5.73813,5.71184,5.73056,5.73223,5.73361,5.78678,1255_g_at


In [ ]:
expr_annot

,ID,Gene Symbol,Gene Title,GSM5916425,GSM5916426,GSM5916427,GSM5916428,GSM5916429,GSM5916430,GSM5916431,GSM5916432,GSM5916433,GSM5916434,GSM5916435,GSM5916436,GSM5916437,GSM5916438,GSM5916439
0,1007_s_at,DDR1 /// MIR4640,discoidin domain receptor tyrosine kinase 1 //...,8.25153,7.76315,8.40313,7.98926,8.05217,8.44563,7.58667,8.02614,9.79436,9.66555,9.49759,9.89471,8.45370,8.85845,8.84316
1,1053_at,RFC2,"replication factor C (activator 1) 2, 40kDa",6.74268,6.75010,6.50660,6.88312,6.97066,6.76910,6.89180,6.67075,6.93478,6.90036,6.62057,6.62056,6.52978,6.72519,6.55780
2,117_at,HSPA6,heat shock 70kDa protein 6 (HSP70B'),6.74961,6.53426,6.72746,6.90891,6.80198,6.80546,6.76402,6.76341,6.74732,6.86814,6.90635,6.84800,6.63013,6.73454,6.55409
3,121_at,PAX8,paired box 8,8.79223,8.73107,9.32590,9.22579,9.28703,9.42747,9.04310,8.86239,9.51909,9.22565,9.39310,9.32185,9.42897,8.99391,8.99834
4,1255_g_at,GUCA1A,guanylate cyclase activator 1A (retina),5.60172,5.69342,5.70023,5.77163,5.73856,5.75393,5.76626,5.64828,5.76027,5.73813,5.71184,5.73056,5.73223,5.73361,5.78678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54670,AFFX-ThrX-5_at,NaN,NaN,5.64099,5.67748,5.70707,5.77138,5.75951,5.75188,5.51195,5.60986,5.79810,5.73435,5.77338,5.76123,5.77051,5.78763,5.80817
54671,AFFX-ThrX-M_at,NaN,NaN,5.75442,5.74797,5.61645,5.79415,5.77837,5.74816,5.72737,5.74435,5.72464,5.70966,5.72084,5.66925,5.79708,5.78825,5.59525
54672,AFFX-TrpnX-3_at,NaN,NaN,5.54234,5.60438,5.63702,5.64852,5.58681,5.59516,5.58031,5.58455,5.59259,5.62689,5.57209,5.57667,5.56555,5.60404,5.67943
54673,AFFX-TrpnX-5_at,NaN,NaN,5.85795,6.03098,5.78590,6.01309,5.85351,5.88400,6.06078,5.96022,5.87353,5.85035,5.83316,5.79860,5.95485,5.85965,5.91752


In [ ]:
expr_gene.to_csv("../data/GSE/GSE197406_gene_expression_mean.csv")

##### now we do the DEGs analysis using python ##

In [ ]:
import numpy as np
pheno["condition"] = np.where(
    pheno["source_name_ch1"].str.contains("no liver abnormality",
                                          case=False,
                                          na=False),
                                          "control","treated"
)

In [ ]:
pheno[["geo_accession","source_name_ch1","condition"]]
pheno["condition"].value_counts()

condition
control    8
treated    7
Name: count, dtype: int64